In [55]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.genai import types
from google.adk.tools import google_search
from google.genai import types
import asyncio
import pprint
import sys

from google.adk.tools.agent_tool import AgentTool

from datetime import datetime

import json
from typing import Dict, Any

import os


In [56]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
model_name = "gemini-2.0-flash"

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

short_config = types.GenerationConfig(
    max_output_tokens=500, # Limit response to 500 tokens
)

model_config = Gemini(
    model=model_name,
    retry_options=retry_config,
    generation_config=short_config,
    api_key=GOOGLE_API_KEY,
)

In [71]:
instructions = """
    You are a helpful shopping assistant. Use the tools provided to find products, compare prices, 
    and provide recommendations based on user preferences.
    When using the google_search tool, be specific with your queries to get the best results. 
    Focus on finding high-quality products and good deals from UK based retailers.
    
    IMPORTANT: You MUST return EXACTLY 6 product recommendations, no more, no less.
    Select the 3 best options based on price, quality, and availability.

    Return with brief descriptions and prices in following JSON format:
    [
        {
            "seller": str = "Seller Name",
            "brand": str = "Brand Name",
            "product": str = "Product Name",
            "price": float = "Product Price",
            "user_review_score": float = "User Review Score"
        },
        ...
    ]
"""

shopping_agent = Agent(
    model=model_config,
    name="shopping_agent",
    description="An agent for shopping assistance",
    tools=[google_search],
    output_key="shopping_recommendations",
    instruction=instructions,
)
print("Shopping agent created.")


Shopping agent created.


In [58]:
instructions = """
    You are a sustainability evaluation agent. Use the tools provided to research companies,
    focusing on their environmental impact, ethical practices, and sustainability initiatives.

    You will review a list of brands from the shopping_recommendations. Evaluate their sustainability
    based on recent data. You need to use the google_search tool to find relevant information.
    
    Your key evaluation results should include:
    - Environmental Impact: Assess the brand's carbon footprint, resource usage, and waste management.
    - Ethical Practices: Evaluate labor practices, fair trade certifications, and community engagement.
    - Animal Welfare: Consider policies on animal testing and cruelty-free certifications.
    - Sustainability Initiatives: Look for programs or policies aimed at reducing environmental impact.

    For each of those categories make a score between 1 and 5, where 1 is poor and 5 is excellent.
    
    Scorring rules:
    - 1: Very poor performance, significant negative impact or unethical practices.
    - 2: Below average performance, some negative impact or questionable practices.
    - 3: Average performance, mention of ethical practices on company website but with no external validation.
    - 4: Good performance, mention of ethical practices on company website with some external validation.
    - 5: Excellent performance, mention of ethical practices on company website with strong external validation.
    
    Combine these scores to provide an overall sustainability rating for each brand.
    
    For each brand, return a brief evaluation in the following JSON format:
    [
        {
            "brand": str = "Brand Name",
            "environment_impact_score": int = 1-5,
            "ethical_practices_score": int = 1-5,
            "animal_welfare_score": int = 1-5,
            "sustainability_initiatives_score": int = 1-5,
            "overall_sustainability_rating": float = 1-5,
            "summary": str = "Brief summary of the evaluation for each of the categories and
                justification for the overall rating",
        },
        ...
    ]
"""

sustainability_evaluation_agent = Agent(
    model=model_config,
    name="sustainability_evaluation_agent",
    description="An agent for sustainability evaluation",
    tools=[google_search],
    output_key="sustainability_evaluation",
    instruction=instructions,
)
print("Sustainability evaluation agent created.")


Sustainability evaluation agent created.


In [59]:
instructions = """
    You are a sustainable shopping assistant. Your task is to:
    
    1. First, call the shopping_agent to get product recommendations
    2. Then, call the sustainability_evaluation_agent with the brands from those recommendations
    3. Finally, combine both results to make a final recommendation
    
    Provide the user with product recommendations that balance quality, price, and sustainability.
    For each product recommendation, include the sustainability evaluation of the brand.
    
    Make single final recommendation for the best product in the following JSON format:
    {
            "brand": str = "Brand Name",
            "environment_impact_score": int = 1-5,
            "ethical_practices_score": int = 1-5,
            "animal_welfare_score": int = 1-5,
            "sustainability_initiatives_score": int = 1-5,
            "overall_sustainability_rating": float = 1-5,
            "user_review_score": float = "User Review Score"
            "summary": str = "Brief summary of the evaluation for each of the categories and
                justification for the overall rating",
    }
    Sustainability evaluation should be prioritized when making final recommendations.
"""

root_agent = Agent(
    model=model_config,
    name="sustainable_shopping_assistant",
    description="An agent that assists users in making sustainable shopping choices",
    tools=[AgentTool(agent=shopping_agent), AgentTool(agent=sustainability_evaluation_agent)],
    instruction=instructions,
)

print("Sustainable shopping assistant created.")


Sustainable shopping assistant created.


In [64]:
def display_recommendation(response):
    """Display results in a user-friendly format"""
    
    # Extract final recommendation
    final_text = None
    for event in reversed(response):
        if event.content and event.content.parts:
            for part in reversed(event.content.parts):
                if hasattr(part, 'text') and part.text:
                    final_text = part.text
                    break
            if final_text:
                break
    
    if not final_text:
        print("No recommendation found")
        return
    
    # Parse JSON
    try:
        if "```json" in final_text:
            start = final_text.find("```json") + 7
            end = final_text.find("```", start)
            final_text = final_text[start:end].strip()
        
        recommendation = json.loads(final_text)
    except json.JSONDecodeError:
        print("Could not parse recommendation as JSON")
        print(final_text)
        return
    
    # Display header
    print("   SUSTAINABLE SHOPPING RECOMMENDATION - PRODUCT DETAILS")
    
    # Product Information
    print("─" * 60)
    print(f"  Brand:    {recommendation.get('brand', 'N/A')}")
        
    # Sustainability Scores
    print("\nSUSTAINABILITY SCORES")
    print("─" * 60)
    
    env_score = recommendation.get('environment_impact_score', 0)
    eth_score = recommendation.get('ethical_practices_score', 0)
    ani_score = recommendation.get('animal_welfare_score', 0)
    sus_score = recommendation.get('sustainability_initiatives_score', 0)
    overall = recommendation.get('overall_sustainability_rating', 0.0)
    
    print(f"  Environmental Impact:        {env_score}/5")
    print(f"  Ethical Practices:           {eth_score}/5")
    print(f"  Animal Welfare:              {ani_score}/5")
    print(f"  Sustainability Initiatives:  {sus_score}/5")


    # Color-coded overall rating
    if overall >= 4.0:
        rating_text = "EXCELLENT"
    elif overall >= 3.0:
        rating_text = "GOOD"
    elif overall >= 2.0:
        rating_text = "FAIR"
    else:
        rating_text = "POOR"

    print(f"\n  OVERALL SUSTAINABILITY:      {overall:.1f}/5   {rating_text}")
    
    
    # Summary
    print("\nEVALUATION SUMMARY")
    print("─" * 60)
    summary = recommendation.get('summary', 'No summary available')
    # Word wrap summary
    import textwrap
    wrapped = textwrap.fill(summary, width=58, initial_indent="  ", subsequent_indent="  ")
    print(wrapped)


In [61]:
def display_all_responses(response):
    """Display all function calls, responses, and final outputs from the agent run"""
    
    # Print all responses
    for i, event in enumerate(response):
        content = event.content
        
        # Check for function calls
        for part in content.parts:
            if part.function_call:
                function_call = part.function_call
                print(f"\n[Event {i}] Function Call: {function_call.name}")
                print("-" * 60)
                pprint.pp(function_call.args["request"], depth=1)
            
            if part.function_response:
                function_response = part.function_response
                print(f"\n[Event {i}] Function Response: {function_response.name}")
                print("-" * 60)
                pprint.pp(function_response.response["result"], depth=1)
            
            # THIS IS THE MISSING PART - Root agent's text output
            if hasattr(part, 'text') and part.text:
                print(f"\n[Event {i}] Text Response (Root Agent)")
                print("-" * 60)
                print(part.text)
        
        # Display token usage
        metadata = event.usage_metadata
        if metadata:
            tokens = metadata.total_token_count
            print(f"\n[Event {i}] Tokens used: {tokens}")
            print("="*60)


In [62]:
async def query(query_text: str, display_all_responses_flag=False):
    """Run the agent with logging to a file"""

    # Check if runner exists globally, if not create it
    global runner
    if 'runner' not in globals() or runner is None:
        print("Initializing runner...")
        runner = InMemoryRunner(
            agent=root_agent,
            plugins=[LoggingPlugin()]
        )
        print("Runner initialized")

    old_stdout = sys.stdout

    # Use timestamp for unique log files
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = open(f"agent_logs_{timestamp}.log", "w", encoding="utf-8")
    sys.stdout = log_file

    response = await runner.run_debug(query_text)

    sys.stdout = old_stdout
    log_file.close()

    display_recommendation(response)

    if display_all_responses_flag:
        display_all_responses(response)


In [73]:
await query("Toiletries under £30")
await query("dog food")

   SUSTAINABLE SHOPPING RECOMMENDATION - PRODUCT DETAILS
────────────────────────────────────────────────────────────
  Brand:    NIVEA

SUSTAINABILITY SCORES
────────────────────────────────────────────────────────────
  Environmental Impact:        4/5
  Ethical Practices:           3/5
  Animal Welfare:              3/5
  Sustainability Initiatives:  4/5

  OVERALL SUSTAINABILITY:      3.5/5   GOOD

EVALUATION SUMMARY
────────────────────────────────────────────────────────────
  NIVEA demonstrates a good understanding of its
  environmental impact and has several initiatives to
  reduce it. They are committed to reducing CO2 emissions,
  using less fossil-based virgin plastic, and ensuring
  their packaging is recyclable, reusable, or refillable
  by 2025. They aim to source renewable raw and packaging
  materials from sustainable and deforestation-free
  sources. They do not test on animals themselves, but
  they sell products in countries where animal testing is
  required by law